In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch #First you do a Zero shot Bert sentiment classification so when manually labelling the sentiment, it will be less time consuming as you only need to correct a few and only need to check whether BERT has done it right
import requests
import re

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [3]:
tokens = tokenizer.encode('I have cancelled mine', return_tensors='pt')
result = model(tokens)
result.logits
int(torch.argmax(result.logits))+1

1

In [4]:
# Making an function for the Bert model, the number will first be changed to an integer from 1 to 5
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [5]:
import pandas as pd
import numpy as np

df_dutch = pd.read_csv("dutch_comments_filtered_combined.csv")
df_english = pd.read_csv("english_comments_filtered_combined.csv")

print(df_dutch.head())
print(df_english.head())

       Car Model Language     Video ID         Author          Published At  \
0  Tesla Model Y    Dutch  E5mfhe-Q6lE  @BartHuitsing  2023-06-26T11:38:09Z   
1  Tesla Model Y    Dutch  E5mfhe-Q6lE  @werner134897  2023-03-11T06:08:18Z   
2  Tesla Model Y    Dutch  E5mfhe-Q6lE    @ronnie9187  2023-01-17T15:07:13Z   
3  Tesla Model Y    Dutch  E5mfhe-Q6lE      @jote2275  2022-11-06T15:39:27Z   
4  Tesla Model Y    Dutch  E5mfhe-Q6lE      @Maszzmic  2022-01-16T20:44:26Z   

             Updated At  Like Count  \
0  2023-06-26T11:38:09Z           1   
1  2023-03-11T06:10:00Z           0   
2  2023-01-17T15:07:13Z           0   
3  2022-11-06T15:39:27Z           0   
4  2022-01-16T20:44:26Z           0   

                                        Comment Text  \
0  Eén van de belangrijkste voordelen van de Y in...   
1  Nadruk op hogere prijs en gewicht is mijns inz...   
2  Het valt me op dat er lakschade aan de voorkan...   
3  Electrische auto&#39;s zijn enorm groot en duu...   
4  <a href

In [6]:
# BERT does not work with NA therefore NA will be checked and replaced.
missing_dutch = df_dutch['Cleaned Comment Text'].isna().sum()
missing_english = df_english['Cleaned Comment Text'].isna().sum()

print(f"Missing values in Dutch comments: {missing_dutch}")
print(f"Missing values in English comments: {missing_english}")


Missing values in Dutch comments: 0
Missing values in English comments: 25


In [7]:
df_dutch['sentiment_score'] = df_dutch['Cleaned Comment Text'].apply(
    lambda x: sentiment_score(x[:512]) if isinstance(x, str) else None
)

df_english['sentiment_score'] = df_english['Cleaned Comment Text'].apply(
    lambda x: sentiment_score(x[:512]) if isinstance(x, str) else None
)


In [28]:
df_english.to_csv("english_comments_with_sentiment_score.csv", index=False)
df_dutch.to_csv("dutch_comments_with_sentiment_score.csv", index=False)

In [9]:
print(df_dutch.head())
print(df_english.head(10))

       Car Model Language     Video ID         Author          Published At  \
0  Tesla Model Y    Dutch  E5mfhe-Q6lE  @BartHuitsing  2023-06-26T11:38:09Z   
1  Tesla Model Y    Dutch  E5mfhe-Q6lE  @werner134897  2023-03-11T06:08:18Z   
2  Tesla Model Y    Dutch  E5mfhe-Q6lE    @ronnie9187  2023-01-17T15:07:13Z   
3  Tesla Model Y    Dutch  E5mfhe-Q6lE      @jote2275  2022-11-06T15:39:27Z   
4  Tesla Model Y    Dutch  E5mfhe-Q6lE      @Maszzmic  2022-01-16T20:44:26Z   

             Updated At  Like Count  \
0  2023-06-26T11:38:09Z           1   
1  2023-03-11T06:10:00Z           0   
2  2023-01-17T15:07:13Z           0   
3  2022-11-06T15:39:27Z           0   
4  2022-01-16T20:44:26Z           0   

                                        Comment Text  \
0  Eén van de belangrijkste voordelen van de Y in...   
1  Nadruk op hogere prijs en gewicht is mijns inz...   
2  Het valt me op dat er lakschade aan de voorkan...   
3  Electrische auto&#39;s zijn enorm groot en duu...   
4  <a href

In [26]:
print(df_dutch.iloc[486])

Car Model                                                      Volvo XC40
Language                                                            Dutch
Video ID                                                      __QoKqx3MlE
Author                                                          @Jokesxxl
Published At                                         2024-06-02T18:32:04Z
Updated At                                           2024-06-02T18:32:04Z
Like Count                                                              1
Comment Text            Wat een saaie bak dit, koop dan een Alfa Stelv...
Cleaned Comment Text    wat een saai bak dit koop dan een alfa stelvio...
sentiment_score                                                         1
Name: 486, dtype: object
